In [1]:
import scipp as sc
import numpy as np
import matplotlib.pyplot as plt
import plopp as pp
from scipp import curve_fit
from scipy.special import voigt_profile

%matplotlib widget

## In this example, we will look at a sample that has an elastic signal from incoherent scattering (from the sample and the sample holder), and a quasi-elastic magnetic signal from the sample. The goal is to determine the width and area of the magnetic signal as function of Q. In principle, we also want to look at temperature dependence, but we skip that for now.

## Data analysis steps
# 1. Load and inspect data
# 2. Load and inspect vanadium data
# 3. Define resolution function and fit vanadium data
# 4. Define fit function and fit data
# 5. Plot data and fit
# 6. Define fit function for fitted parameters, fit parameters and plot




In [64]:
## Load and inspect data
# For the GUI, this should ideally be a selection menu, where multiple data files can be loaded into one dataset. I'd like us to automatically determine what the coordinates are, e.g. temperature, magnetic field etc.
# Several data sets (jobs?) should be allowed. The user should be able to select which data set to work with.
# Users should also be allowed to make a mask for data, e.g. to remove NaNs or data at positions with detector problems or similar.

#TODO Make a general data loader together with SCIPP team
## Load actual data - just one temperature for now
NUMBER_OF_Q_POINTS=16
NUMBER_OF_E_POINTS=1024

#TODO consider adding a loop to load more temperatures

#Preallocate
intensity_values=np.zeros((NUMBER_OF_Q_POINTS,NUMBER_OF_E_POINTS))
error_values=np.zeros((NUMBER_OF_Q_POINTS,NUMBER_OF_E_POINTS))

# Load data into a matrix
for Q in range(NUMBER_OF_Q_POINTS):
    filename = '../../IN16b_GGG_data/data_450mK_Q' +str(Q+1) +'.dat'

    data_array = np.loadtxt(filename)
    energy_values=data_array[:, 0] #should be the same for all Q
    # EnergyValues[Q,:]=data_array[:, 0]
    intensity_values[Q,:]=data_array[:,1]
    error_values[Q,:]=data_array[:,2]

# Define energy, Q and intensity as scipp variables with units, and make a DataArray
energy=sc.array(dims=['energy'],values=energy_values/1000,unit='meV')
Q=sc.array(dims=['Q'],values=[  0.5708,    0.7002,    0.8262 ,   0.9485 ,   1.0664  ,  1.1793   , 1.2868 ,   1.3883 ,   1.4833 ,   1.5716  ,  1.6525  ,  1.7258  ,  1.7910 ,   1.8480  ,  1.8965 ,   1.9361],unit='1/angstrom')
intensity=sc.array(dims=['Q','energy'],values=intensity_values,variances=error_values*error_values) #The variance is the square of the uncertainty!

GGG_data_450mK = sc.DataArray(data=intensity, coords={'Q':Q,'energy': energy})

#######################
#Preallocate
intensity_values=np.zeros((NUMBER_OF_Q_POINTS,NUMBER_OF_E_POINTS))
error_values=np.zeros((NUMBER_OF_Q_POINTS,NUMBER_OF_E_POINTS))

# Load data into a matrix
for Q in range(NUMBER_OF_Q_POINTS):
    filename = '../../IN16b_GGG_data/data_600mK_Q' +str(Q+1) +'.dat'

    data_array = np.loadtxt(filename)
    energy_values=data_array[:, 0] #should be the same for all Q
    # EnergyValues[Q,:]=data_array[:, 0]
    intensity_values[Q,:]=data_array[:,1]
    error_values[Q,:]=data_array[:,2]

# Define energy, q and intensity as scipp variables with units, and make a DataArraw
energy=sc.array(dims=['energy'],values=energy_values/1000,unit='meV')
Q=sc.array(dims=['Q'],values=range(NUMBER_OF_Q_POINTS))
intensity=sc.array(dims=['Q','energy'],values=intensity_values,variances=error_values*error_values) #The variance is the square of the uncertainty!

GGG_data_600mK = sc.DataArray(data=intensity, coords={'Q':Q,'energy': energy})

# Bin the data
#TODO consider if grouping is better than binning
Q_BINS=16
ENERGY_BINS=sc.scalar(1e-3*0.2, unit='meV')

INTENSITY_MIN=0.0
INTENSITY_MAX=0.01

ENERGY_MIN = -0.03  * sc.Unit('meV')
ENERGY_MAX = 0.03 * sc.Unit('meV')

# da.coords['Energy'] = da.coords['Energy'].to(unit='micro*eV') #optional change the scale to mueV
binned_GGG_data_450mK=GGG_data_450mK.flatten(to='dummy').bin(energy=ENERGY_BINS,Q=Q_BINS).bins.mean() #can add .plot() to plot it
binned_GGG_data_600mK=GGG_data_600mK.flatten(to='dummy').bin(energy=ENERGY_BINS,Q=Q_BINS).bins.mean() #can add .plot() to plot it
pp.slicer(binned_GGG_data_450mK['energy',ENERGY_MIN:ENERGY_MAX],vmin=INTENSITY_MIN,vmax=INTENSITY_MAX,
     keep=['energy'],
     linestyle='none',
     marker='o',
     markerfacecolor='none',
     color='k'
)
# sc.show(GGG_data_450mK)

# The slicer can also slice the data along more coordinates, such as temperature. Users should be allowed to select which coordinates to keep, as well as the intensity and energy range.

Box(children=(InteractiveFig(children=(HBar(), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(icon…

ValueError: No finite values were found in array. Cannot compute limits.

ValueError: No finite values were found in array. Cannot compute limits.

In [4]:
# Just testing the 600 mK data. Note that it has not been binned, so the background is very noisy.
pp.slicer(GGG_data_600mK['energy',ENERGY_MIN:ENERGY_MAX],vmin=INTENSITY_MIN,vmax=INTENSITY_MAX,
     keep=['energy'],
     linestyle='none',
     marker='o',
     markerfacecolor='none',
     color='k'
)



Box(children=(InteractiveFig(children=(HBar(), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(icon…

In [5]:
## 2. Load and inspect vanadium data

#Preallocate
intensity_values=np.zeros((NUMBER_OF_Q_POINTS,NUMBER_OF_E_POINTS))
error_values=np.zeros((NUMBER_OF_Q_POINTS,NUMBER_OF_E_POINTS))

# Load data into a matrix
for Q in range(NUMBER_OF_Q_POINTS):
    filename = '../../IN16b_GGG_data/vanadium_Q' +str(Q+1) +'.dat'

    data_array = np.loadtxt(filename)
    energy_values=data_array[:, 0] #should be the same for all Q
    # EnergyValues[Q,:]=data_array[:, 0]
    intensity_values[Q,:]=data_array[:,1]
    error_values[Q,:]=data_array[:,2]

# Define energy, q and intensity as scipp variables with units, and make a DataArray
energy=sc.array(dims=['energy'],values=energy_values/1000,unit='meV')
Q=sc.array(dims=['Q'],values=range(NUMBER_OF_Q_POINTS))
intensity=sc.array(dims=['Q','energy'],values=intensity_values,variances=error_values*error_values) #The variance is the square of the uncertainty!

vanadium_data = sc.DataArray(data=intensity, coords={'Q':Q,'energy': energy})

## Bin and plot data 
Q_BINS=16
ENERGY_BINS=sc.scalar(1e-3*0.2, unit='meV')

INTENSITY_MIN_VANADIUM=0.0
INTENSITY_MAX_VANADIUM=0.06

ENERGY_MIN_VANADIUM = -0.02 * sc.Unit('meV')
ENERGY_MAX_VANADIUM = 0.02 * sc.Unit('meV')

# da.coords['Energy'] = da.coords['Energy'].to(unit='micro*eV') #optional change the scale to mueV
binned_vanadium_data=vanadium_data.flatten(to='dummy').bin(energy=ENERGY_BINS,Q=Q_BINS).bins.mean() #can add .plot() to plot it

pp.slicer(binned_vanadium_data['energy',ENERGY_MIN_VANADIUM:ENERGY_MAX_VANADIUM],vmin=INTENSITY_MIN_VANADIUM,vmax=INTENSITY_MAX_VANADIUM,
     keep=['energy'],
     linestyle='none',
     marker='o',
     markerfacecolor='none',
     color='k'
)


Box(children=(InteractiveFig(children=(HBar(), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(icon…

In [6]:
## 3. Define resolution function and fit vanadium data
# TODO: Make a library of functions instead of defining them inline.
# TODO: Add a Lorentzian tail to the resolution function
# The data is approximately Gaussian with Lorentzian tails. Those are not important for a quick fit, but should be added.


def resolution_function_scipp(energy, energy_offset, res_gauss1_area, res_gauss1_sigma, res_BG):
    """
    Calculate the resolution function using the given parameters.

    Parameters:
    energy (array-like): The energy values.
    energy_offset (float): The energy offset.
    res_gauss1_area (float): The area of the first Gaussian component.
    res_gauss1_sigma (float): The standard deviation of the first Gaussian component.
    res_BG (float): The background value.

    Returns:
    array-like: The calculated resolution function values.
    """
    x = energy
    x = x - energy_offset

    y = res_gauss1_area * 1 / np.sqrt(2 * np.pi) / res_gauss1_sigma * sc.exp(-0.5 * (x / res_gauss1_sigma) ** 2) + res_BG
    return y

# Fit the resolution function to the vanadium data

# TODO Add the option of providing start guess for each Q point, e.g. by using the value from higher or lower Q, or a different temperature.
popt, _ = curve_fit(['energy'], resolution_function_scipp, vanadium_data, 
                    p0 = {'res_gauss1_area' : 5e-5*sc.Unit('meV'), 
                          'res_gauss1_sigma': 4e-4 * sc.Unit('meV'),
                          'energy_offset'   : 0 * sc.Unit('meV'),
                          'res_BG'          :1e-5})

#Calculate the fit to display it
#TODO Make it possible to calculate the fit on a more dense set of points than the data
vanadium_fit=resolution_function_scipp(vanadium_data.coords['energy'],
                                      energy_offset     =sc.values(popt['energy_offset']),
                                      res_gauss1_area   =sc.values(popt['res_gauss1_area']),
                                      res_gauss1_sigma  =sc.values(popt['res_gauss1_sigma']),
                                      res_BG            =sc.values(popt['res_BG']))
vanadium_fit.coords['energy']=energy


# Make a group of the data and the fit for the slicer
data_and_fit=sc.DataGroup({'Data': binned_vanadium_data,
                            'Fit': vanadium_fit})

pp.slicer(data_and_fit['energy',ENERGY_MIN:ENERGY_MAX],vmin=INTENSITY_MIN,vmax=INTENSITY_MAX,
     keep=['energy'],
     linestyle=         {'Data': 'none',    'Fit': '-'},
     marker=            {'Data': 'o',       'Fit':'none'},
     markerfacecolor=   {'Data': 'none',    'Fit':'red'},
     color=             {'Data': 'black',   'Fit':'red'}
)

# Of course, the user should be allowed to change the colors, markers and so on.

Box(children=(InteractiveFig(children=(HBar(), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(icon…

In [7]:
## 4. Define fit function and fit data

# TODO: Add a Lorentzian tail to the resolution function
# Define the fit function, which for now is just an elastic peak, a Lorentzian peak and a background. The resolution function is fixed to the Gaussian one determined from the vanadium data.
def GGG_fit_function_scipp(energy, energy_offset, res_gauss1_sigma,el_area,lorz1_area,lorz1_HWHM,BG):
    """
    Elastic plus quasielastic signal with a Gaussian resolution and a constant background

    Parameters:
    energy (array-like): The energy values.
    energy_offset (float): The energy offset.
    res_gauss1_sigma (float): The standard deviation of the Gaussian resolution function.
    el_area (float): The area of the elastic peak.
    lorz1_area (float): The area of the Lorentzian peak.
    lorz1_HWHM (float): The half-width at half-maximum of the Lorentzian peak.
    BG (float): The background value.

    Returns:
    array-like: The calculated GGG fit function values.
    """

    # Subtract energy offset from energy values
    x = energy-energy_offset

    # Calculate elastic peak using resolution function
    y_el = resolution_function_scipp(energy, energy_offset, el_area, res_gauss1_sigma, 0)

    # scipp doesn't allow using some functions like the voigt_profile, imported from scipy. We therefore need to convert the scipp arrays to numpy arrays.
    # TODO Make a scipp version of the voigt_profile
    y_lorz=lorz1_area.value*(voigt_profile(x.values, res_gauss1_sigma.value, lorz1_HWHM.value))

    # Sum elastic and Lorentzian peaks and add background
    y = y_el.values + y_lorz + BG.values

    # Return the result as a scipp array
    y=sc.array(dims=['energy'],values=y,unit='dimensionless')    
    
    return y

# Define the initial guess for the fit parameters
energy_offset= -0.5e-3 * sc.Unit('meV')
res_gauss1_sigma=4e-4 * sc.Unit('meV')
el_area=1e-5*sc.Unit('meV')
lorz1_area=1e-5*sc.Unit('meV')
lorz1_HWHM=1e-3*sc.Unit('meV')
BG=0.5e-3 *sc.Unit('dimensionless')


# Each Q point has its own set of start guess. For now, we just use the same values for all Q points, but later, the user should be allowed to set these independently for each Q point.
energy_offset_array=sc.ones(dims=['Q'], shape=[NUMBER_OF_Q_POINTS])*energy_offset
res_gauss1_sigma_array=sc.ones(dims=['Q'], shape=[NUMBER_OF_Q_POINTS])*res_gauss1_sigma
el_area_array=sc.ones(dims=['Q'], shape=[NUMBER_OF_Q_POINTS])*el_area
lorz1_area_array=sc.ones(dims=['Q'], shape=[NUMBER_OF_Q_POINTS])*lorz1_area
lorz1_HWHM_array=sc.ones(dims=['Q'], shape=[NUMBER_OF_Q_POINTS])*lorz1_HWHM
BG_array=sc.ones(dims=['Q'], shape=[NUMBER_OF_Q_POINTS])*BG

# Calculate the guess; this is useful for the user to determine if the guess is reasonable
GGG_intensity_guess=sc.zeros(dims=['Q','energy'],shape=[NUMBER_OF_Q_POINTS,NUMBER_OF_E_POINTS])
for k in range(NUMBER_OF_Q_POINTS):
    GGG_intensity_guess['Q',k]=GGG_fit_function_scipp(energy, energy_offset_array[k], res_gauss1_sigma_array[k],el_area_array[k],lorz1_area_array[k],lorz1_HWHM_array[k],BG_array[k])
GGG_guess_450mK=sc.DataArray(data=GGG_intensity_guess, coords={'Q':Q,'energy': energy})





# The user should be allowed to fix some parameters, for example all parameters related to the resolution
# Fit the data. We can add more functionality, such as minimum and maximum values, but for now we keep it simple.
popt_GGG, _ = curve_fit(['energy'], GGG_fit_function_scipp, GGG_data_450mK, 
                    p0 = {
                        'energy_offset'   : energy_offset,
                        'res_gauss1_sigma':res_gauss1_sigma,
                        'el_area'         : el_area, 
                        'lorz1_area'      : lorz1_area, 
                        'lorz1_HWHM'      : lorz1_HWHM, 
                        'BG'              : BG
                        })

# Calculate the fit in the same way as the guess was calculated
GGG_450mK_fit_values=sc.zeros(dims=['Q','energy'],shape=[NUMBER_OF_Q_POINTS,NUMBER_OF_E_POINTS])
for k in range(NUMBER_OF_Q_POINTS):
    GGG_450mK_fit_values['Q',k]=GGG_fit_function_scipp(energy, sc.values(popt_GGG['energy_offset'])[k], sc.values(popt_GGG['res_gauss1_sigma'])[k],sc.values(popt_GGG['el_area'])[k],sc.values(popt_GGG['lorz1_area'])[k],sc.values(popt_GGG['lorz1_HWHM'])[k],sc.values(popt_GGG['BG'])[k])
GGG_fit_450mK=sc.DataArray(data=GGG_450mK_fit_values, coords={'Q':Q,'energy': energy})

# Make a group of the data, guess and fitfor the slicer
# For the user, it should be possible to show/hide the data, guess and fit
data_and_guess_and_fit=sc.DataGroup({'Data': binned_GGG_data_450mK,
                                     'Guess': GGG_guess_450mK,
                                     'Fit': GGG_fit_450mK})

data_and_fit_slicer=pp.slicer(data_and_guess_and_fit['energy',ENERGY_MIN:ENERGY_MAX],vmin=INTENSITY_MIN,vmax=INTENSITY_MAX,
     keep=['energy'],
     linestyle=         {'Data': 'none', 'Guess':'-' ,   'Fit': '-'},
     marker=            {'Data': 'o',    'Guess':'none' ,'Fit':'none'},
     markerfacecolor=   {'Data': 'none', 'Guess':'green','Fit':'red'},
     color=             {'Data': 'black','Guess':'green','Fit':'red'}
)
#TODO Add a button to show/hide the guess and fit
#TODO Change the label of the y axis
data_and_fit_slicer


Box(children=(InteractiveFig(children=(HBar(), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(icon…

In [9]:

p=sc.plot(popt_GGG['lorz1_HWHM'][1:15].to(unit='micro*eV'))
p.canvas.ylabel = 'HWHM [µeV]'
p.canvas.xrange = [0, 15]
p


InteractiveFig(children=(HBar(), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(icon='home', layou…

In [62]:
p=sc.plot(popt_GGG['lorz1_area'][1:15].to(unit='micro*eV'))
p.canvas.ylabel = 'Area [µeV]'
p.canvas.xrange = [0, 15]
p
#Notice a peak near 4 and 11. If I had used proper units for Q, these peaks would be meaningful.

InteractiveFig(children=(HBar(), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(icon='home', layou…

In [59]:
#7. Define fit function for fitted parameters, fit parameters and plot
# The simplest possible fit function
def str_line(Q, a,b):
    y=a*Q+b
    return y

#Define the data to be fitted
lorz1_HWHM_fit_result=popt_GGG['lorz1_HWHM'][1:14].to(unit='micro*eV')

popt_HWHM, _ = curve_fit(['Q'], str_line, lorz1_HWHM_fit_result, 
                    p0 = {
                        'a'   : 0*sc.Unit('micro*eV'),
                        'b'   : 2.7*sc.Unit('micro*eV')
                        })

Q=sc.array(dims=['Q'],values=range(NUMBER_OF_Q_POINTS))
lorz1_HWHM_strline_fit_value=str_line(Q[1:14], sc.values(popt_HWHM['a']), sc.values(popt_HWHM['b']))
lorz1_HWHM_strline_fit_value.coords['Q']=Q[1:14]
fit_result_for_plot=sc.DataGroup({'Data': lorz1_HWHM_fit_result,
                                     'Fit': lorz1_HWHM_strline_fit_value})

p=sc.plot(fit_result_for_plot)
p.canvas.ylabel = 'HWHM [µeV]'
p.canvas.xrange = [0, 15]
p

#Users should be able to fit any parameters from the previous fits, along any coordinate and to any function, such as the ones implemented in QENSLibrary.
# Here I have changed the units to microeV, that kind of functionality should also be available to the user.

InteractiveFig(children=(HBar(), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(icon='home', layou…